In [1]:
from os import listdir
from os.path import isfile, join
import re

In [2]:
path = 'keyboards/windows'
filenames = listdir(path)
filenames.remove('_platform.xml')

In [3]:
modifiers = {}
for fn in filenames:
    with open(join(path, fn)) as fh:
        for line in fh:
            if '<keyMap' in line:
                line = line.strip()
                line = line.replace('+caps?', '')
                line = line.replace('+shift?', '')
                line = line.replace('+alt?', '')
                if line not in modifiers:
                    modifiers[line] = []
                modifiers[line].append(fn)

In [4]:
{k:len(v) for k,v in modifiers.items()}

{'<keyMap>': 208,
 '<keyMap modifiers="shift">': 204,
 '<keyMap modifiers="caps">': 146,
 '<keyMap modifiers="caps+shift">': 148,
 '<keyMap modifiers="altR ctrl+alt">': 151,
 '<keyMap modifiers="ctrl">': 174,
 '<keyMap modifiers="altR+shift ctrl+alt+shift">': 79,
 '<keyMap modifiers="ctrl+shift">': 27,
 '<keyMap modifiers="shift caps">': 4,
 '<keyMap modifiers="altR ctrl">': 2}

In [5]:
for kb in filenames:
    kb = kb.replace('-t-k0-windows', '')[:-4]
    print(kb)

dsb-extended
tr-legacy
en-var
iu
as
jv
bg-Latn
de
te
la-Ital
dsb-legacy
lis-extended
ga
es-419
km
yo
zh-MO
pt-PT
gd
tg
ga-Ogam
mn-Phag
he-extended
kn
ky
zh-Hant
az-Cyrl
uk
lt-lt1205
el-Latn
es-extended
chr
cs-qwerty
mn
mt
ar
bs-Cyrl
en-dvorakl
fr-BE
ro-extended
or
non
ig
sl
ka-qwerty
ta
en-extended
ka-legacy
mt-101key
hi
kk
dv
ur
bg
man
bg-qwerty
tt-legacy
nso
nl-BE
fr-CA-var
lv-extended
haw
tzm
pt-var
de-BE-var
ka-var
th-var
si-legacy
bo-extended
lb
wo
el
syr-qwerty
chr-phonetic
et
uk-extended
fr-BE-var
dz
hi-legacy
pa
sk-qwerty
cs
ug-legacy
tr
sk
bm
en-GB
fi
mr
fa
lo
mi
en
mn-Mong-extended
kl
so-Osma
sat-Olck
pl-extended
bug
az-extended
iu-var
he
dsb
lv-qwerty
ja
cs-extended
fr-CH
fr-CA-legacy
se-FI
de-CH
is
de-BE
th
bg-legacy
ka-legacy-var
ar-azerty
bn
hy
en-GB-extended
it-var
dv-legacy
hu-101key
km-extended
my
en-dvorakr
sah
tmh-extended
pt
ne
hu
ro-legacy
en-dvorak
mk-legacy
bo
ru
si
gu
th-patta-var
lt-lt1582
gn
ps
ru-phonetic
it
mk
el-el319
srb
hy-legacy
el-POLYTON
el-Latn-el220


In [6]:
import xml.etree.ElementTree as ET

keyboard_map_dict = {}
for fn in filenames:
    kb = fn.replace('-t-k0-windows', '')[:-4]
    tree = ET.parse(join(path, fn))
    name = tree.find('names/name').attrib['value']
    keymaps = tree.findall('keyMap')
    keyboard_map_dict[kb] = {'lang': name}
    for km in keymaps:
        att = km.attrib
        parse_km = False
        if 'modifiers' in att:
            mod = att['modifiers']
            mod = mod.replace('+caps?', '').replace('+shift?', '').replace('+alt?', '')
            if mod in {'shift', 'altR ctrl+alt', 'altR+shift ctrl+alt+shift'}:
                parse_km = True
        else:
            mod = 'none'
            parse_km = True
        mod = mod.split()[0].replace('altR', 'altgr')

        if parse_km:
            mod_dict = {}
            maps = km.getchildren()
            for k in maps:
                to = k.attrib['to']
                mo = True
                while mo:
                    mo = re.search(r'\\u{([0-9A-F]+)}', to)
                    if mo:
                        to = to.replace(mo.group(0), chr(int(mo.group(1), 16)))
                mod_dict[k.attrib['iso']] = to
            keyboard_map_dict[kb][mod] = mod_dict
print(keyboard_map_dict)

{'dsb-extended': {'lang': 'Sorbian Extended', 'none': {'E00': '^', 'E01': '1', 'E02': '2', 'E03': '3', 'E04': '4', 'E05': '5', 'E06': '6', 'E07': '7', 'E08': '8', 'E09': '9', 'E10': '0', 'E11': 'ß', 'E12': '´', 'D01': 'q', 'D02': 'w', 'D03': 'e', 'D04': 'r', 'D05': 't', 'D06': 'z', 'D07': 'u', 'D08': 'i', 'D09': 'o', 'D10': 'p', 'D11': 'ü', 'D12': '+', 'C01': 'a', 'C02': 's', 'C03': 'd', 'C04': 'f', 'C05': 'g', 'C06': 'h', 'C07': 'j', 'C08': 'k', 'C09': 'l', 'C10': 'ö', 'C11': 'ä', 'C12': 'ł', 'B00': '<', 'B01': 'y', 'B02': 'x', 'B03': 'c', 'B04': 'v', 'B05': 'b', 'B06': 'n', 'B07': 'm', 'B08': ',', 'B09': '.', 'B10': '-', 'A03': ' '}, 'shift': {'E00': '°', 'E01': '!', 'E02': '"', 'E03': '§', 'E04': '$', 'E05': '%', 'E06': '&', 'E07': '/', 'E08': '(', 'E09': ')', 'E10': '=', 'E11': '?', 'E12': '`', 'D01': 'Q', 'D02': 'W', 'D03': 'E', 'D04': 'R', 'D05': 'T', 'D06': 'Z', 'D07': 'U', 'D08': 'I', 'D09': 'O', 'D10': 'P', 'D11': 'Ü', 'D12': '*', 'C01': 'A', 'C02': 'S', 'C03': 'D', 'C04': 'F'

/home/chrislit/fast/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


In [7]:
len(keyboard_map_dict)

208

In [8]:
import json

In [9]:
for kb in keyboard_map_dict:
    with open('keymaps/{}.json'.format(kb), 'w') as fp:
        json.dump(keyboard_map_dict[kb], fp)